In [1]:
import pandas as pd
import sqlite3

In [2]:
orders=pd.read_csv("/content/orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users=pd.read_json("/content/users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()

In [7]:
with open("/content/restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

In [8]:
restaurants = pd.read_sql(
    "SELECT * FROM restaurants",
    conn
)

restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [9]:
merged_data = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)

In [10]:
final_df = pd.merge(
    merged_data,
    restaurants,
    how="left",
    on="restaurant_id"
)

In [11]:
final_df.to_csv(
    "/content/final_food_delivery_dataset.csv",
    index=False
)

In [12]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df.groupby(final_df['order_date'].dt.month).size()

/tmp/ipython-input-3432000186.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


,0
order_date,
1,831
2,785
3,903
4,812
5,844
6,784
7,859
8,851
9,812


In [21]:
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [22]:
final_df.groupby('city')['total_amount'].sum()

,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Hyderabad,1889366.58
Pune,1924797.93


In [23]:
final_df.groupby('membership')['total_amount'].sum()

,total_amount
membership,
Gold,3975364.89
Regular,4036259.23


In [24]:
final_df.groupby('cuisine')['total_amount'].sum()

,total_amount
cuisine,
Chinese,1930504.65
Indian,1971412.58
Italian,2024203.80
Mexican,2085503.09


In [25]:
final_df.groupby('restaurant_name_y')['total_amount'].sum()

,total_amount
restaurant_name_y,
Restaurant_1,12083.73
Restaurant_10,19230.11
Restaurant_100,20971.56
Restaurant_101,13798.00
Restaurant_102,20101.12
...,...
Restaurant_95,12357.72
Restaurant_96,22209.66
Restaurant_97,17064.40


In [26]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

final_df.groupby(final_df['order_date'].dt.month)['total_amount'].sum()

,total_amount
order_date,
1,663020.26
2,630867.40
3,716738.98
4,666097.18
5,668428.61
6,610822.93
7,688559.45
8,694987.58
9,653838.07


In [27]:
final_df = final_df.drop(columns=['restaurant_name_x'])
final_df = final_df.rename(columns={'restaurant_name_y': 'restaurant_name'})

In [28]:
gold_city_revenue = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [29]:
cuisine_avg_order = (
    final_df
    .groupby('cuisine')['total_amount']
    .mean()
    .sort_values(ascending=False)
)

cuisine_avg_order

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [30]:
high_value_users = (
    final_df
    .groupby('user_id')['total_amount']
    .sum()
)

(high_value_users > 1000).sum()

np.int64(2544)

In [31]:
# Create rating bins
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']

final_df['rating_range'] = pd.cut(
    final_df['rating'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Total revenue per rating range
rating_revenue = (
    final_df
    .groupby('rating_range')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

rating_revenue

/tmp/ipython-input-2241380518.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('rating_range')['total_amount']


,total_amount
rating_range,
4.6-5.0,2197030.75
3.0-3.5,2136772.70
4.1-4.5,1960326.26
3.6-4.0,1717494.41


In [32]:
gold_city_avg = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .mean()
    .sort_values(ascending=False)
)

gold_city_avg

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [33]:
cuisine_analysis = (
    final_df
    .groupby('cuisine')
    .agg(
        distinct_restaurants=('restaurant_id', 'nunique'),
        total_revenue=('total_amount', 'sum')
    )
    .sort_values(['distinct_restaurants', 'total_revenue'],
                 ascending=[True, False])
)

cuisine_analysis

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [36]:
total_orders = len(final_df)

gold_orders = len(final_df[final_df['membership'] == 'Gold'])

percentage_gold_orders = round((gold_orders / total_orders) * 100)

percentage_gold_orders

50

In [37]:
restaurant_analysis = (
    final_df
    .groupby('restaurant_name')
    .agg(
        total_orders=('order_id', 'count'),
        avg_order_value=('total_amount', 'mean')
    )
)

# Filter restaurants with less than 20 orders
filtered_restaurants = restaurant_analysis[
    restaurant_analysis['total_orders'] < 20
]

# Sort by highest average order value
filtered_restaurants.sort_values(
    'avg_order_value',
    ascending=False
)

,total_orders,avg_order_value
restaurant_name,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [38]:
combo_revenue = (
    final_df
    .groupby(['membership', 'cuisine'])['total_amount']
    .sum()
    .sort_values(ascending=False)
)

combo_revenue

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [39]:
# Ensure order_date is datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

# Create quarter column
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

# Revenue per quarter
quarter_revenue = (
    final_df
    .groupby('quarter')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [40]:
gold_orders_count = final_df[final_df['membership'] == 'Gold'].shape[0]
gold_orders_count

4987

In [41]:
hyderabad_revenue = (
    final_df[final_df['city'] == 'Hyderabad']['total_amount']
    .sum()
    .round()
)

hyderabad_revenue

np.float64(1889367.0)

In [42]:
distinct_users = final_df['user_id'].nunique()
distinct_users

2883

In [43]:
avg_order_value_gold = (
    final_df[final_df['membership'] == 'Gold']['total_amount']
    .mean()
    .round(2)
)

avg_order_value_gold

np.float64(797.15)

In [44]:
high_rating_orders = final_df[final_df['rating'] >= 4.5].shape[0]
high_rating_orders

3374

In [45]:
# Step 1: Filter Gold members
gold_df = final_df[final_df['membership'] == 'Gold']

# Step 2: Find top revenue city among Gold members
top_gold_city = (
    gold_df
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

top_gold_city

'Chennai'

In [46]:
# Step 3: Count orders in that city (Gold members only)
orders_top_gold_city = gold_df[gold_df['city'] == top_gold_city].shape[0]
orders_top_gold_city

1337

In [47]:
set(orders.columns).intersection(set(users.columns))

{'user_id'}

In [48]:
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [49]:
len(orders), len(final_df)

(10000, 10000)

In [50]:
final_df.isna().sum()

,0
order_id,0
user_id,0
restaurant_id,0
order_date,0
total_amount,0
name,0
city,0
membership,0
restaurant_name,0
cuisine,0


In [51]:
pd.merge(orders, users, on='user_id', how='left')

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [52]:
users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [53]:
set(orders.columns).intersection(set(restaurants.columns))

{'restaurant_id', 'restaurant_name'}

In [54]:
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [55]:
final_df['user_id'].value_counts().head()

,count
user_id,
2973,13
1515,12
874,11
496,11
1337,11
